In [79]:
import import_ipynb
import numpy as np
import cv2
import random
import copy
from DataAugment import data_augment
import threading
import itertools
import math
from keras.layers import Input

In [87]:
def iou(box1,box2):     #box shape like [x1,y1,x2,y2]
    x1=max(box1[0],box2[0])
    y1=max(box1[1],box2[1])
    x2=min(box1[2],box2[2])
    y2=min(box1[3],box2[3])
    intersection_area=abs((x1-x2)*(y1-y2))
    box1_area = abs((box1[2] - box1[0]) * (box1[3] - box1[1]))
    box2_area = abs((box2[2] - box2[0]) * (box2[3] - box2[1]))
    union_area = box1_area + box2_area - intersection_area
    iou =intersection_area / (union_area+1e-6)
    return iou

In [88]:
def new_img_size(width, height, img_min_side=600):
    if width <= height:
        k = float(img_min_side) / width
        resized_height = int(k * height)
        resized_width = img_min_side
    else:
        k = float(img_min_side) / height
        resized_width = int(k * width)
        resized_height = img_min_side

    return resized_width, resized_height

In [89]:
from pretrained_model import pretrained_resnet
def base_layer(inp):
    #inp = Input(shape = (None,None,3))
    base_network = pretrained_resnet(inp)
    base_out = base_network.layers[-1].output
    return base_out
#i_inp = Input(shape = (None,None,3))
#roi_inp = Input(shape = (None,4))
#s_layers = base_layer(i_inp)

In [90]:
#def rpn(img_data,w,h,resized_w,resized_h,img_length_calc_function):
def rpn(img_data,w,h,resized_w,resized_h):
    downscale=float(16)
    anc_sizes=[128, 256, 512]
    anc_ratios=[[1, 1], [1./math.sqrt(2), 2./math.sqrt(2)], [2./math.sqrt(2), 1./math.sqrt(2)]]
    num_anc=9
    #(out_w, out_h) = img_length_calc_function(resized_w, resized_h)
    i_inp = Input(shape = (resized_h,resized_w,3))
    base_out= base_layer(i_inp)
    out_w,out_h=base_out.shape[2],base_out.shape[1]
    num_ratios=3
    y_rpn_overlap=np.zeros((out_h,out_w,num_anc))
    y_is_box_valid=np.zeros((out_h,out_w,num_anc))
    y_rpn_regr=np.zeros((out_h,out_w,4*num_anc))
    num_bbox=len(img_data['bboxes'])
    num_anc_for_bbox = np.zeros((num_bbox)).astype(int)
    best_anc_for_bbox = -1*np.ones((num_bbox, 4)).astype(int)
    best_iou_for_bbox = np.zeros((num_bbox)).astype(np.float32)
    best_x_for_bbox = np.zeros((num_bbox, 4)).astype(int)
    best_dx_for_bbox = np.zeros((num_bbox, 4)).astype(np.float32)
    gtc=np.zeros((num_bbox,4))
    for b_num,bbox in enumerate(img_data['bboxes']):
        gtc[b_num,0]=bbox['x1']*resized_w/w
        gtc[b_num,1]=bbox['x2']*resized_w/w
        gtc[b_num,2]=bbox['y1']*resized_h/h
        gtc[b_num,3]=bbox['y2']*resized_h/h
    for anc_s in range(3):
        for anc_r in range(3):
            anc_w=anc_sizes[anc_s]*anc_ratios[anc_r][0]
            anc_h=anc_sizes[anc_s]*anc_ratios[anc_r][1]
            for i in range(out_w):
                x1_anc = downscale * (i + 0.5) - anc_w / 2
                x2_anc = downscale * (i + 0.5) + anc_w / 2
                if x1_anc<0 or x2_anc>resized_w:
                    continue
                for j in range(out_h):
                    y1_anc=downscale*(j+0.5)-anc_h/2
                    y2_anc=downscale*(j+0.5)+anc_h/2
                    if y1_anc<0 or y2_anc>resized_h:
                        continue
                    best_iou_for_pos=0.0
                    bbox_type='neg'
                    for bbox in range(num_bbox):
                        k=img_data['bboxes'][bbox]['class']
                        now_iou=iou([x1_anc,y1_anc,x2_anc,y2_anc],[gtc[b_num,0],gtc[b_num,2],gtc[b_num,1],gtc[b_num,3]])
                        if now_iou>best_iou_for_bbox[bbox] or now_iou>0.7:
                            cx=(gtc[b_num,0]+gtc[b_num,1])/2
                            cy=(gtc[b_num,2]+gtc[b_num,3])/2
                            cxa=x1_anc+x2_anc/2
                            cya=y1_anc+y2_anc/2
                            tx=(cx-cxa)/x2_anc-x1_anc
                            ty=(cy-cya)/y2_anc-y1_anc
                            tw=np.log((gtc[b_num,1]-gtc[b_num,0])/(x2_anc-x1_anc))
                            th=np.log((gtc[b_num,3]-gtc[b_num,2])/(y2_anc-y1_anc))
                        if(k!='bg'):
                            if now_iou>best_iou_for_bbox[bbox]:
                                best_iou_for_bbox[bbox]=now_iou
                                best_anc_for_bbox[bbox]=[j,i,anc_r,anc_s]
                                best_x_for_bbox[bbox]=[x1_anc,x2_anc,y1_anc,y2_anc]
                                best_dx_for_bbox[bbox]=[tx,ty,tw,th]
                            if now_iou>0.7:
                                bbox_type='pos'
                                num_anc_for_bbox[bbox]+=1
                                if now_iou>best_iou_for_pos:
                                    best_iou_for_pos=now_iou
                                    best_regr=[tx,ty,tw,th]
                            if 0.3<now_iou<0.7:
                                if bbox_type!='pos':
                                    bbox_type='neutral'
                    if bbox_type=='neg':
                        y_is_box_valid[j,i,anc_r+num_ratios*anc_s]=1
                        y_rpn_overlap[j,i,anc_r+num_ratios*anc_s]=0
                    elif bbox_type=='neutral':
                        y_is_box_valid[j,i,anc_r+num_ratios*anc_s]=0
                        y_rpn_overlap[j,i,anc_r+num_ratios*anc_s]=0
                    elif bbox_type=='pos':
                        y_is_box_valid[j,i,anc_r+num_ratios*anc_s]=1
                        y_rpn_overlap[j,i,anc_r+num_ratios*anc_s]=1
                        initial = 4 * (anc_r + num_ratios * anc_s)
                        y_rpn_regr[j, i, initial:initial+4] = best_regr
    
    for l in range(num_anc_for_bbox.shape[0]):
        if num_anc_for_bbox[l]==0:
            if best_anc_for_bbox[l,0]==-1:
                continue
            y_is_box_valid[best_anc_for_bbox[l,0], best_anc_for_bbox[l,1], best_anc_for_bbox[l,2] + num_ratios *best_anc_for_bbox[l,3]] = 1
            y_rpn_overlap[best_anc_for_bbox[l,0], best_anc_for_bbox[l,1], best_anc_for_bbox[l,2] + num_ratios *best_anc_for_bbox[l,3]] = 1                        
            initial = 4 * (best_anc_for_bbox[l,2] + num_ratios * best_anc_for_bbox[l,3])
            y_rpn_regr[best_anc_for_bbox[l,0], best_anc_for_bbox[l,1], initial:initial+4] = best_dx_for_bbox[l, :]
                    
    y_rpn_overlap=np.expand_dims(np.transpose(y_rpn_overlap,(2,0,1)),axis=0)
    y_is_box_valid=np.expand_dims(np.transpose(y_is_box_valid,(2,0,1)),axis=0)
    y_rpn_regr=np.expand_dims(np.transpose(y_rpn_regr,(2,0,1)),axis=0)
    
    
    #pos_site=np.where(np.logical_and(y_rpn_overlap[0,:,:,:](np.equal(y_rpn_overlap[0,:,:,:],1)),y_is_box_valid[0,:,:,:](np.equal(y_is_box_valid[0,:,:,:],1))))
    #neg_site=np.where(np.logical_and(y_rpn_overlap[0,:,:,:](np.equal(y_rpn_overlap[0,:,:,:],0)),y_is_box_valid[0,:,:,:](np.equal(y_is_box_valid[0,:,:,:],1))))
    pos_site = np.where(np.logical_and(y_rpn_overlap[0, :, :, :] == 1, y_is_box_valid[0, :, :, :] == 1))
    neg_site = np.where(np.logical_and(y_rpn_overlap[0, :, :, :] == 0, y_is_box_valid[0, :, :, :] == 1))
    
    if(len(pos_site[0])>128):
        invalid=random.sample(range(len(pos_site[0])),len(pos_site[0])-128)
        y_is_box_valid[0, pos_site[0][invalid], pos_site[1][invalid], pos_site[2][invalid]] = 0
    if len(neg_site[0])>128:
        invalid = random.sample(range(len(neg_site[0])), len(neg_site[0]) - 128)
        y_is_box_valid[0, neg_site[0][invalid], neg_site[1][invalid], neg_site[2][invalid]] = 0
    
    y_rpn_cls=np.concatenate([y_is_box_valid,y_rpn_overlap],axis=1)
    y_rpn_regr=np.concatenate([np.repeat(y_rpn_overlap, 4, axis=1), y_rpn_regr],axis=1)
    
    
    return np.copy(y_rpn_cls),np.copy(y_rpn_regr)
    

In [91]:
ybv=2*np.ones((1,9,3,2))
yro=3*np.ones((1,9,3,2))
np.shape(np.concatenate([ybv,yro],axis=1))
np.shape(np.transpose(np.concatenate([ybv,yro],axis=1),(0,2,3,1)))

(1, 3, 2, 18)

In [92]:
#def anch_rpn(all_img_data, class_count,img_length_calc_function,mode='train'):
def anch_rpn(all_img_data, class_count,mode='train'):
    sample_selector = SampleSelector(class_count)
    while True:
        if mode=='train':
            np.random.shuffle(all_img_data)
        for img_data in all_img_data:
            
                if sample_selector.skip_sample_for_balanced_class(img_data):
                    continue
                if mode=='train':
                    img_data_aug,x_img=data_augment(img_data,augment=True)
                else:
                    img_data_aug,x_img=data_augment(img_data,augment=False)
                (w,h)=(img_data_aug['width'],img_data_aug['height'])
                (rows,cols,_)=x_img.shape
                assert cols==w
                assert rows==h
                (resized_w,resized_h)=new_img_size(w,h,600)
                x_img=cv2.resize(x_img,(resized_w,resized_h),interpolation=cv2.INTER_CUBIC)
                #y_rpn_cls, y_rpn_regr =rpn(img_data_aug, w, h, resized_w, resized_h, img_length_calc_function)
                y_rpn_cls, y_rpn_regr =rpn(img_data_aug, w, h, resized_w, resized_h)
                x_img=x_img[:,:,(2,1,0)].astype(np.float32)
                img_channel_mean = [103.939, 116.779, 123.68]
                x_img[:,:,0]-=   img_channel_mean[0]
                x_img[:,:,1]-=   img_channel_mean[1]
                x_img[:,:,2]-=   img_channel_mean[2]
                x_img=np.transpose(x_img,(2,0,1))
                x_img=np.expand_dims(x_img,axis=0)
                y_rpn_regr[:, y_rpn_regr.shape[1]//2:, :, :] *= 4
                x_img = np.transpose(x_img, (0, 2, 3, 1))
                y_rpn_cls = np.transpose(y_rpn_cls, (0, 2, 3, 1))
                y_rpn_regr = np.transpose(y_rpn_regr, (0, 2, 3, 1))
                
                #yield x_img, [y_rpn_cls, y_rpn_regr], img_data_aug
                yield np.copy(x_img), [np.copy(y_rpn_cls), np.copy(y_rpn_regr)], img_data_aug
       
    

In [93]:
class SampleSelector:
    def __init__(self, class_count):
        self.classes = [b for b in class_count.keys() if class_count[b] > 0]
        self.class_cycle = itertools.cycle(self.classes)
        self.curr_class = next(self.class_cycle)
    def skip_sample_for_balanced_class(self, img_data):
        class_in_img = False
        for bbox in img_data['bboxes']:
            cls_name = bbox['class']
            if cls_name == self.curr_class:
                class_in_img = True
                self.curr_class = next(self.class_cycle)
                break
        if class_in_img:
            return False
        else:
            return True           

In [69]:
#from Dataextract import data

In [70]:
#all_imgs, cls_count, cls_map  =data(r'C:\Users\ANSHUL\Downloads\VOCdevkit')

In [68]:
#all_imgs[0]
#b[0][0].shape
#k=np.where(b[0][0][:,:,:]==1)
#print(k)

In [67]:
#def get_img_output_length(width, height):
#    def get_output_length(input_length):
#        # zero_pad
#        input_length += 6
#        # apply 4 strided convolutions
#        filter_sizes = [7, 3, 1, 1]
#        stride = 2
#        for filter_size in filter_sizes:
#            input_length = (input_length - filter_size + stride) // stride
#        return input_length

  #  return get_output_length(width), get_output_length(height)

In [65]:
#a,b,c=next(anch_rpn(all_imgs, cls_count,mode='train'))

In [34]:
#import cv2
#cv2.imshow('img',a[0])
#cv2.waitKey(0)
#cv2.destroyAllWindows()